In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)


In [5]:
custody_df = pd.read_csv("../data/CUSTODY_Dividend_bookings 1.csv", sep=";")
NBIM_df = pd.read_csv("../data/NBIM_Dividend_bookings 1.csv", sep=";")

In [6]:
custody_df[custody_df["COAC_EVENT_KEY"] == 950123456].transpose()

,0
COAC_EVENT_KEY,950123456
ISIN,US0378331005
EVENT_EX_DATE,07.02.2025
EVENT_PAYMENT_DATE,14.02.2025
CUSTODY,501234567
SEDOL,2046251
CUSTODIAN,CUST/JPMORGANUS
EVENT_TYPE,DVCA
NOMINAL_BASIS,1500000
LOAN_QUANTITY,0


In [7]:
NBIM_df[NBIM_df["COAC_EVENT_KEY"] == 950123456].transpose()

,0
COAC_EVENT_KEY,950123456
INSTRUMENT_DESCRIPTION,APPLE INC
ISIN,US0378331005
SEDOL,2046251
TICKER,AAPL
ORGANISATION_NAME,Apple Inc
DIVIDENDS_PER_SHARE,0.25
EXDATE,07.02.2025
PAYMENT_DATE,14.02.2025
CUSTODIAN,JPMORGAN_CHASE


In [8]:
# Map columns that have the same meaning but different names
column_mapping = {
    "COAC_EVENT_KEY": "COAC_EVENT_KEY",
    "ISIN": "ISIN",
    "SEDOL": "SEDOL",
    "NOMINAL_BASIS": "NOMINAL_BASIS",
    "GROSS_AMOUNT": "GROSS_AMOUNT_QUOTATION",
    "NET_AMOUNT_QC": "NET_AMOUNT_QUOTATION",
    "NET_AMOUNT_SC": "NET_AMOUNT_SETTLEMENT",
    "TAX": "WTHTAX_COST_QUOTATION",
    "TAX_RATE": "WTHTAX_RATE",
    "DIV_RATE": "DIVIDENDS_PER_SHARE",
    "EX_DATE": "EXDATE",
    "PAY_DATE": "PAYMENT_DATE",
    "CUSTODY": "BANK_ACCOUNT",
    "SETTLED_CURRENCY": "SETTLEMENT_CURRENCY",
    # Add more mappings as needed
}

# Find shared columns (by mapping)
shared_cols_custody = []
shared_cols_nbim = []
for c_custody, c_nbim in column_mapping.items():
    if c_custody in custody_df.columns and c_nbim in NBIM_df.columns:
        shared_cols_custody.append(c_custody)
        shared_cols_nbim.append(c_nbim)

# Merge on COAC_EVENT_KEY to align events
merged = pd.merge(
    custody_df,
    NBIM_df,
    left_on="COAC_EVENT_KEY",
    right_on="COAC_EVENT_KEY",
    suffixes=("_custody", "_nbim"),
    how="outer",
    indicator=True
)

# For each shared column, compare values and report inconsistencies
inconsistencies = []

for c_custody, c_nbim in zip(shared_cols_custody, shared_cols_nbim):
    col_custody = c_custody + "_custody" if c_custody != "COAC_EVENT_KEY" else c_custody
    col_nbim = c_nbim + "_nbim" if c_nbim != "COAC_EVENT_KEY" else c_nbim

    if col_custody not in merged.columns or col_nbim not in merged.columns:
        continue

    # Compare values (handle NaNs and floats)
    for idx, row in merged.iterrows():
        val_custody = row[col_custody]
        val_nbim = row[col_nbim]
        # Use np.isclose for floats, else direct comparison
        if pd.isnull(val_custody) and pd.isnull(val_nbim):
            continue
        elif pd.isnull(val_custody) or pd.isnull(val_nbim):
            inconsistencies.append({
                "COAC_EVENT_KEY": row["COAC_EVENT_KEY"],
                "Column_Custody": c_custody,
                "Value_Custody": val_custody,
                "Column_NBIM": c_nbim,
                "Value_NBIM": val_nbim,
                "Reason": "Value missing in one dataframe"
            })
        else:
            # Try float comparison if possible
            try:
                if np.issubdtype(type(val_custody), np.number) or np.issubdtype(type(val_nbim), np.number):
                    if not np.isclose(float(val_custody), float(val_nbim), atol=1e-2, equal_nan=True):
                        inconsistencies.append({
                            "COAC_EVENT_KEY": row["COAC_EVENT_KEY"],
                            "Column_Custody": c_custody,
                            "Value_Custody": val_custody,
                            "Column_NBIM": c_nbim,
                            "Value_NBIM": val_nbim,
                            "Reason": "Numeric mismatch"
                        })
                else:
                    if str(val_custody).strip() != str(val_nbim).strip():
                        inconsistencies.append({
                            "COAC_EVENT_KEY": row["COAC_EVENT_KEY"],
                            "Column_Custody": c_custody,
                            "Value_Custody": val_custody,
                            "Column_NBIM": c_nbim,
                            "Value_NBIM": val_nbim,
                            "Reason": "String mismatch"
                        })
            except Exception as e:
                if str(val_custody).strip() != str(val_nbim).strip():
                    inconsistencies.append({
                        "COAC_EVENT_KEY": row["COAC_EVENT_KEY"],
                        "Column_Custody": c_custody,
                        "Value_Custody": val_custody,
                        "Column_NBIM": c_nbim,
                        "Value_NBIM": val_nbim,
                        "Reason": f"Comparison error: {e}"
                    })

# Display inconsistencies as a DataFrame
inconsistencies_df = pd.DataFrame(inconsistencies)
if not inconsistencies_df.empty:
    display(inconsistencies_df)
else:
    print("No inconsistencies found in shared columns.")


,COAC_EVENT_KEY,Column_Custody,Value_Custody,Column_NBIM,Value_NBIM,Reason
0,970456789,NOMINAL_BASIS,20000,NOMINAL_BASIS,15000,Numeric mismatch
1,970456789,NOMINAL_BASIS,20000,NOMINAL_BASIS,10000,Numeric mismatch
2,970456789,NOMINAL_BASIS,30000,NOMINAL_BASIS,20000,Numeric mismatch
3,970456789,NOMINAL_BASIS,30000,NOMINAL_BASIS,15000,Numeric mismatch
4,970456789,NOMINAL_BASIS,30000,NOMINAL_BASIS,10000,Numeric mismatch
5,970456789,NOMINAL_BASIS,10000,NOMINAL_BASIS,20000,Numeric mismatch
6,970456789,NOMINAL_BASIS,10000,NOMINAL_BASIS,15000,Numeric mismatch


In [9]:
# Assume column_mapping is defined above as a dict: {custody_col: nbim_col}
# If not, you may need to define it based on your join logic.
analysis_key = 960789012

# Select the row for COAC_EVENT_KEY == 950123456 from each DataFrame
row_custody = custody_df[custody_df["COAC_EVENT_KEY"] == analysis_key]
row_nbim = NBIM_df[NBIM_df["COAC_EVENT_KEY"] == analysis_key]

# If multiple rows, take the first (should only be one for this event)
row_custody = row_custody.iloc[0] if not row_custody.empty else pd.Series(dtype=object)
row_nbim = row_nbim.iloc[0] if not row_nbim.empty else pd.Series(dtype=object)

# Build reverse mapping for NBIM -> Custody
reverse_column_mapping = {v: k for k, v in column_mapping.items() if v is not None}

# Get all unique fields from both sources
fields_custody = set(row_custody.index)
fields_nbim = set(row_nbim.index)
all_fields = sorted(fields_custody.union(fields_nbim))

data = []
for field in all_fields:
    # Determine mapping
    if field in column_mapping:
        mapped_field = column_mapping[field]
        mapping_type = "Custody"
    elif field in reverse_column_mapping:
        mapped_field = reverse_column_mapping[field]
        mapping_type = "NBIM"
    else:
        mapped_field = None
        mapping_type = None

    # Get values
    if mapping_type == "Custody":
        val_custody = row_custody[field] if field in row_custody else None
        val_nbim = row_nbim[mapped_field] if mapped_field in row_nbim else None if mapped_field else None
        other_column = mapped_field if mapped_field != field else None
    elif mapping_type == "NBIM":
        val_custody = row_custody[mapped_field] if mapped_field in row_custody else None if mapped_field else None
        val_nbim = row_nbim[field] if field in row_nbim else None
        other_column = mapped_field if mapped_field != field else None
    else:
        val_custody = row_custody[field] if field in row_custody else None
        val_nbim = row_nbim[field] if field in row_nbim else None
        other_column = None

    # For comparison, treat both as string for simplicity, but handle None
    identical = (val_custody == val_nbim) if (val_custody is not None and val_nbim is not None) else True
    data.append([field, val_custody, val_nbim, identical, other_column])

df_compare = pd.DataFrame(data, columns=["Field", "Custody", "NBIM", "Identical", "Other_Column"])
display(df_compare)

,Field,Custody,NBIM,Identical,Other_Column
0,ADR_FEE,0,None,True,None
1,ADR_FEE_RATE,0,None,True,None
2,AVG_FX_RATE_QUOTATION_TO_PORTFOLIO,None,0.008234,True,None
3,BANK_ACCOUNT,712345678,712345678,True,CUSTODY
4,BANK_ACCOUNTS,712345678,None,True,None
5,COAC_EVENT_KEY,960789012,960789012,True,None
6,CURRENCIES,KRW USD,None,True,None
7,CUSTODIAN,CUST/HSBCKR,HSBC_KOREA,False,None
8,CUSTODY,712345678,712345678,True,BANK_ACCOUNT
9,DIVIDENDS_PER_SHARE,361.0,361.0,True,DIV_RATE
